# LCO/LPO

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index, get_disorientation
from numpy import array, dot, round, cross, eye
from numpy.linalg import inv, det, norm

/Users/jasonxie/.local/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
from pymatgen.core.structure import Structure

In [3]:
my_interface = core('cif_files/LCO/CONTCAR',\
                    'cif_files/LPO_2/CONTCAR')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [4]:
#target disorientation
R = get_disorientation(L1 = my_interface.conv_lattice_1, L2 = my_interface.conv_lattice_2, \
                       v1 = [0,1,0], hkl1 = [1, 0, 4], \
                       v2 = [1,0,0], hkl2 = [0, 0, 1])
R

array([[-5.00025777e-01, -4.98503885e-01,  7.08144123e-01],
       [ 8.66010521e-01, -2.87828435e-01,  4.08877206e-01],
       [-2.86161570e-06,  8.17709403e-01,  5.75631246e-01]])

In [27]:
#target disorientation
R = get_disorientation(L1 = my_interface.conv_lattice_1, L2 = my_interface.conv_lattice_2, \
                       v1 = [0,1,0], hkl1 = [1, 0, 4], \
                       v2 = [-1.,  2., -1.], hkl2 = [-1, 0, 1])
R

array([[ 0.28686334, -0.70586832, -0.6476568 ],
       [-0.55396209,  0.42934272, -0.71329575],
       [ 0.78155961,  0.56339572, -0.2678616 ]])

In [5]:
#combining two specified surfaces
hkl_1 = get_primitive_hkl([1, 0, 4], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_2, my_interface.lattice_2)

## Searching two_dimensional CSL

In [6]:
help(my_interface.search_fixed)

Help on method search_fixed in module interfacemaster.interface_generator:

search_fixed(R, exact=False, tol=1e-08) method of interfacemaster.interface_generator.core instance
    main loop finding the appx CSL
    
    Parameters
    ----------
    axis : numpy array
        rotation axis
    theta : float
        initial rotation angle, in degree
    theta_range : float
        range varying theta, in degree
    dtheta : float
        step varying theta, in degree



In [33]:
my_interface.parse_limit(du = 2, S  = 0.4, sgm1=10, sgm2=10, dd = 0.5)

#Do searching!
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 3, \
                                    dtheta = 0.01, pre_R=R, pre_dt= True, integer_tol = 1e-8)

Congrates, we found an appx CSL!

U1 = 
[[ 4  0]
 [ 2  7]
 [-1  0]]; sigma_1 = 29

U2 = 
[[ 0  2]
 [-2  0]
 [ 0  0]]; sigma_2 = 4

D = 
[[ 1.08383824  0.08118009 -0.16080132]
 [ 0.08115633  0.9900975  -0.09280504]
 [-0.16078445 -0.09283426  1.263739  ]]

axis = [8.65012022 4.99451639 7.03144803] ; theta = 0.0



## Get bicrystal

In [14]:
my_interface.compute_bicrystal_two_D(hkl_1 = hkl_1, hkl_2=hkl_2, normal_ortho = True, lim = 50, tol_ortho = 1e-3, tol_integer=1e-3)

cell 1:
[[-8 -4  0]
 [-4 -2 11]
 [-1  1  0]]
cell 2:
[[ 0  0 -3]
 [ 0 -3  0]
 [-1  0  0]]


In [16]:
from numpy import *
around(my_interface.bicrystal_U1)

array([[-8., -4., -0.],
       [-4., -2., 11.],
       [-1.,  1.,  0.]])

In [17]:
around(my_interface.bicrystal_U2)

array([[-0.,  0., -3.],
       [ 0., -3.,  0.],
       [-1.,  0., -0.]])

In [18]:
x1 = ceil(5/norm(dot(my_interface.lattice_1,my_interface.bicrystal_U1)[:,0]))
#y1 = ceil(9/norm(dot(my_interface.lattice_1,my_interface.bicrystal_U1)[:,1]))
#z1 = ceil(9/norm(dot(my_interface.lattice_1,my_interface.bicrystal_U1)[:,2]))

In [19]:
x2 = ceil(5/norm(dot(my_interface.lattice_2,my_interface.bicrystal_U2)[:,0]))
#y2 = ceil(9/norm(dot(my_interface.lattice_2,my_interface.bicrystal_U2)[:,1]))
#z2 = ceil(9/norm(dot(my_interface.lattice_2,my_interface.bicrystal_U2)[:,2]))

In [20]:
my_interface.get_bicrystal(two_D = True, xyz_1 = [x1,1,1], xyz_2 = [x2,1,1], dp1 = 0.1, dp2 = 0, dx = 0, vx=1)

In [17]:
norm(my_interface.CNID[:,0])

2.849476333489562

In [18]:
norm(my_interface.CNID[:,1])

5.715762230300908

In [20]:
#To sample the CNID, try
my_interface.sample_CNID(grid = [3,6], xyz_1 = [x1,1,1], xyz_2 = [x2,1,1], dp1 = 0.1, dp2 = 0, dx = -0.5, vx=0.8)

CNID
[[ 0.          1.33333333]
 [-1.          0.66666667]
 [ 0.         -0.33333333]]
making 18 files...
completed


In [133]:
right_1 = array([24.7867897178, 0.9095271859, 13.7594486662])
right_2 = array([37.0678811529, 2.3316922917, 15.2692225735])

In [134]:
middle = ((right_1 + right_2)/2)[1:3]
r_m_y, r_m_z = middle
print(r_m_y, r_m_z)

1.6206097388 14.514335619850002


In [135]:
left_1 = array([22.2413190086999997, 1.4243551294000001, 12.8611351063000008])

In [136]:
left_2 = array([12.1592408800000005, 2.8491354397999999, 11.4306350467000009])

In [137]:
middle = ((left_1 + left_2)/2)[1:3]
l_m_y, l_m_z = middle

In [138]:
print(l_m_y, l_m_z)

2.1367452846 12.1458850765


In [139]:
dsp = array([0, l_m_y - r_m_y, l_m_z - r_m_z])

In [140]:
dsp

array([ 0.        ,  0.51613555, -2.36845054])

In [141]:
with open('lmp.dat','r') as f:
    lines = f.readlines()
    xhi = float(lines[3].split()[1])
    yhi = float(lines[4].split()[1])
    zhi = float(lines[5].split()[1])
    xy = float(lines[6].split()[0])
    xz = float(lines[6].split()[1])
    yz = float(lines[6].split()[2])

In [142]:
atoms = loadtxt('lmp.dat', skiprows=10)

In [143]:
ids = atoms[:,0]
types = atoms[:,1]
atoms = atoms[:,2:5]

In [144]:
atoms[atoms[:,0] > 23] += dsp

In [145]:
with open('lmp_dsp.dat','w') as f:
    f.write('#Head \n')
    f.write('{} atoms \n'.format(len(atoms)))
    f.write('4 atom types \n')
    f.write(f'0 {xhi} xlo xhi \n')
    f.write(f'0 {yhi} ylo yhi \n')
    f.write(f'0 {zhi} zlo zhi \n')
    f.write(f'{xy} {xz} {yz} xy xz yz \n\n')
    f.write('Atoms # atomic\n\n')
    FinalMat = column_stack((ids, types, atoms))
    savetxt(f, FinalMat, fmt='%i %i %.16f %.16f %.16f')
            
            

In [146]:
left_b = 12.0525
cell_min = 12

In [147]:
left_max = 22.3482
right_min = 24.3096
right_max = 37.5451

In [148]:
right_b = right_max + (right_min - left_max) - (left_b - cell_min)

In [149]:
right_b

39.454